In [ ]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import pandas as pd
import sqlalchemy

import sklearn.metrics as metrics
from tensorflow.python.data import Dataset

In [ ]:
#getting movies dataset using Pandas from movie_metadata provided by Prof Zhang
movies_dataframe =  pd.read_csv("movie_metadata.csv", sep=",")

In [ ]:
#getting actors dataset using Pandas from BuzzFeed
actors_dataframe = pd.read_csv("actor-metrics.csv", sep=",")

In [ ]:
#renaming the lead actors column name for each table to have the same name 'lead_actors'
movies_dataframe = movies_dataframe.rename(columns={'actor_1_name': 'lead_actors'})
actors_dataframe = actors_dataframe.rename(columns={'actor': 'lead_actors'})

In [ ]:
#Clean Up Part 1: Remove unnecessary columns
#First eliminate unnecessary columns and then merge, this way the table is easier to look at
#We should remove movies that are too old for us to look at!
movies_dataframe = movies_dataframe[['lead_actors', 'movie_title', 'title_year', 'gross', 'movie_facebook_likes', 'imdb_score']].query('title_year>=2010.0')
actors_dataframe = actors_dataframe[['lead_actors', 'gender', 'race_simple']]

In [ ]:
#Clean Up Part 2: Merge
movies_dataframe_merged = movies_dataframe.merge(actors_dataframe,on='lead_actors')

In [ ]:
#Clean Up Part 3: Remove duplicates
#remove duplicates based on the movie – even though the actors are still duplicated but they worked on different projects with different gross
movies_dataframe_noDuplicates = movies_dataframe_merged.drop_duplicates(subset=['movie_title'], keep='first', inplace=False)

In [ ]:
#Clean Up Part 4: check if we have movies that still have missing gross revenue in this data
movies_dataframe_noDuplicates[np.isnan(movies_dataframe_noDuplicates.gross)]

In [ ]:
#Clean Up Part 4: adding missing revenues to the data
#2 of these are video games – for now the revenue will be replaced with 0
missing_gross = {'Destiny': 0, 'Animals United': 0, 'Ben-Hur':26410477, 'Florence Foster Jenkins': 27383770}
for idx, row in movies_dataframe_noDuplicates[np.isnan(movies_dataframe_noDuplicates.gross)].iterrows():
    movies_dataframe_noDuplicates.at[idx, 'gross'] = missing_gross[row['movie_title'].strip()] 
    #https://stackoverflow.com/questions/10993612/python-removing-xa0-from-string

In [ ]:
#Clean Up Part 4: check one more time if the NaN values are no longer in the data
movies_dataframe_noDuplicates[np.isnan(movies_dataframe_noDuplicates.gross)]

In [ ]:
#Clean Up Part 4: remove video games which now have 0 gross revenue
movies_dataframe_noDuplicates = movies_dataframe_noDuplicates[movies_dataframe_noDuplicates.gross != 0]

In [ ]:
#Clean Up Part 5: check and see movies with 0 Facebook likes
movies_dataframe_noDuplicates.query('movie_facebook_likes == 0')

In [ ]:
#Clean Up Part 5: adding missing Facebook likes to the data
missing_fb_likes = {'The Boss': 339554, 'Larry Crowne': 76200, 'Arthur Christmas': 621630, 'Another Year':3674,
                   'Criminal': 89404, 'By the Sea': 58198, 'Happy Feet 2': 9231151, 'The Big Wedding': 111836,
                   'Green Zone': 224559, 'Margaret': 109, 'Ben-Hur': 244880, 'Dolphin Tale': 470678,
                   'Dolphin Tale 2': 470678, 'The Smurfs 2': 13427249, 'Hope Springs': 194834, 'Florence Foster Jenkins': 43804,
                   'Young Adult': 64561, "Meek's Cutoff": 1272, 'Good Kill': 5903, 'Dinner for Schmucks': 15185,
                   'Freeheld': 52135, 'All Good Things': 45773, 'Rudderless': 28181, 'Cedar Rapids': 5485,
                   "Sarah's Key": 100, 'Mr. Turner': 7058, 'One Direction: This Is Us': 102132, 'That Awkward Moment': 1180067,
                   'The Last Exorcism': 1131959, 'The To Do List': 30926, 'Obvious Child': 51815}
for idx, row in movies_dataframe_noDuplicates.query('movie_facebook_likes == 0').iterrows():
    movies_dataframe_noDuplicates.at[idx, 'movie_facebook_likes'] = missing_fb_likes[row['movie_title'].strip()] 
    

In [ ]:
#Clean Up Part 5: check and see if there are still movies with 0 Facebook likes
movies_dataframe_noDuplicates.query('movie_facebook_likes == 0')

In [ ]:
#Clean Up Part 6: Extended Gathering

def add_movie(df, lead_actors, movie_title, title_year, gross, movie_facebook_likes, imdb_score, gender, race_simple):
    return df.append({'lead_actors' : lead_actors , 'movie_title' : movie_title,
                    'title_year': title_year,'gross': gross,
                    'movie_facebook_likes': movie_facebook_likes, 'imdb_score': imdb_score,
                    'gender': gender, 'race_simple': race_simple}, ignore_index=True)
    

#Adding some new data from 2017 and 2018 (some from 2016)
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Gal Gadot', 'Wonder Woman', 2017, 412563408, 2244745, 7.5, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Gael Garcia Bernal' , 'Coco', 2017, 209726015, 744460, 8.4 ,'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Daniel Kaluuya' , "Get Out", 2017, 176040665, 468828, 7.7, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Anna Kendrick' , "Pitch Perfect 3", 2017 , 104897530, 5961116, 5.9,'female','White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Samuel L. Jackson', "The Hitman's Bodyguard", 2017 , 75468583, 166718, 6.9, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Chadwick Boseman' , 'Black Panther', 2018 , 700059566, 993239 , 7.4, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Sandra Bullock' , "Ocean's 8", 2018 , 139377762, 1031, 6.2, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Denzel Washington', "The Equalizer 2", 2018 , 102084362, 699100, 6.8, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Constance Wu', "Crazy Rich Asians", 2018 , 173765895, 92144, 7.2, 'female', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Margot Robbie', "I, Tonya", 2017 , 30014539, 47220, 7.5, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Storm Reid', "A Wrinkle in Time", 2018 , 100478608, 132133, 4.2, 'female', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'John Boyega', "Pacific Rim Uprising", 2018 , 59587110, 2030862, 5.6, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Taraji P. Henson', 'Hidden Figures', 2016 , 169607287, 623303, 7.8, 'female', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Neel Sethi', 'The Jungle Book', 2016, 364001123, 1235238, 7.4, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Alicia Vikander', 'Tomb Raider', 2018 , 57421715, 220540, 6.4, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Michael B. Jordan', 'Creed II', 2018 , 55806000, 1063852, 8.0, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Timothée Chalamet', 'Call Me by Your Name', 2017 , 18095701, 180986, 7.9, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Daniel Day-Lewis', 'Phantom Thread', 2017 , 21051680, 40359, 7.5, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Gary Oldman', 'Darkest Hour', 2017 , 56468410, 60365, 7.4, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Shally Hawkins', 'The Shape of Water', 2017 , 63859435, 229235, 7.4, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Frances McDormand', 'Three Billboards Outside Ebbing, Missouri', 2017 , 54513740, 99518, 8.2, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Saoirse Ronan', 'Lady Bird', 2017 , 48958273, 127674, 7.5, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Meryl Streep', 'The Post', 2017 , 81903458, 86416, 7.2, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Fionn Whitehead', 'Dunkirk', 2017 , 190068280, 658910, 7.9, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Casey Affleck', 'Manchester by the Sea', 2016 , 47695371, 116732, 7.8, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Andrew Garfield', 'Hacksaw Ridge', 2016 , 67209615, 439250, 8.1, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Ryan Gosling', 'La La Land', 2016 , 151101803, 1100297, 8.0, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Denzel Washington', 'Fences', 2016 , 57682904, 196748, 7.2, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Mahershala Ali', 'Moonlight', 2016 , 27854932, 352818, 7.4, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Amy Adams', 'Arrival', 2016 , 100546139, 187846, 7.9, 'female', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Jeff Bridges', 'Hell or High Water', 2016 , 27007844, 187846, 7.9, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Dev Patel', 'Lion', 2016 , 51738905, 232982, 8.1, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Robert Downey Jr.', 'Avengers: Infinity War', 2018 , 678815482, 14893835, 8.5, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Chris Pratt', 'Jurassic World: Fallen Kingdom', 2018 , 416769345, 9046139, 6.2, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Rami Malek', 'Bohemian Rhapsody', 2018 , 169219170, 437101, 8.4, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Ryan Reynolds', 'Deadpool 2', 2018 , 318491426, 7540994, 7.8, 'male', 'White')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Amandla Stenberg', 'Everything, Everything', 2017 , 34121140, 693540, 6.4, 'female', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Jada Pinkett-Smith', 'Girls Trip', 2017 , 115171585, 345023, 6.2, 'female', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Denzel Washington', 'Roman J. Israel, Esq.', 2017 , 11962778, 49468, 6.4, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'John David Washington', 'BlacKkKlansman', 2018 , 48271960, 76727, 7.6, 'male', 'POC')
movies_dataframe_noDuplicates = add_movie(movies_dataframe_noDuplicates, 'Lady Gaga', 'A Star is Born', 2018 , 195278597, 252728, 8.2, 'female', 'White')
movies_dataframe_noDuplicates

In [ ]:
# Statistical outliers
if(remove_statistical_outliers):
    #Gross
    gross_q25 = movies_dataframe_noDuplicates['gross'].quantile(.25)
    gross_q75 = movies_dataframe_noDuplicates['gross'].quantile(.75)
    gross_interquart_range = gross_q75 - gross_q25
    gross_outer_fence_lower = gross_q25 - (gross_interquart_range * 3)
    gross_outer_fence_higher = gross_q75 + (gross_interquart_range * 3)
    gross_without_outliers = movies_dataframe_noDuplicates.sort_values(by='gross').query('gross Y='  + str(gross_outer_fence_lower)).query('gross <='  + str(gross_outer_fence_higher))

    #Likes
    likes_q25 = movies_dataframe_noDuplicates['movie_facebook_likes'].quantile(.25)
    likes_q75 = movies_dataframe_noDuplicates['movie_facebook_likes'].quantile(.75)
    likes_interquart_range = likes_q75 - likes_q25
    likes_outer_fence_lower = likes_q25 - (likes_interquart_range * 3)
    likes_outer_fence_higher = likes_q75 + (likes_interquart_range * 3)
    likes_without_outliers = movies_dataframe_noDuplicates.sort_values(by='movie_facebook_likes').query('movie_facebook_likes >='  + str(likes_outer_fence_lower)).query('movie_facebook_likes <='  + str(likes_outer_fence_higher))

In [ ]:
# VISUALIZATION
# Part 1: Categorization and data preparation
# First, Get data for sub categories: male & white, male & POC, female & white, female & POC
if(remove_statistical_outliers):
    #likes
    movies_f_likes = likes_without_outliers.query('gender=="female"')
    movies_m_likes = likes_without_outliers.query('gender=="male"')
    movies_w_likes = likes_without_outliers.query('race_simple=="White"')
    movies_p_likes = likes_without_outliers.query('race_simple=="POC"')
    
    movies_fw_likes = movies_w_likes.query('gender=="female"')
    movies_mp_likes = movies_p_likes.query('gender=="male"')
    movies_mw_likes = movies_m_likes.query('race_simple=="White"')
    movies_fp_likes = movies_f_likes.query('race_simple=="POC"')
    
    movies_fw_likes_year = movies_fw_likes.groupby(by='title_year', sort=True)
    movies_mp_likes_year = movies_mp_likes.groupby(by='title_year', sort=True)
    movies_mw_likes_year = movies_mw_likes.groupby(by='title_year', sort=True)
    movies_fp_likes_year = movies_fp_likes.groupby(by='title_year', sort=True)
    
    #sum
    movies_fw_sum_by_year_likes = movies_fw_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    movies_mp_sum_by_year_likes = movies_mp_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    movies_mw_sum_by_year_likes = movies_mw_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    movies_fp_sum_by_year_likes = movies_fp_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    
    #average
    movies_fw_mean_by_year_likes = movies_fw_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    movies_mp_mean_by_year_likes = movies_mp_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    movies_mw_mean_by_year_likes = movies_mw_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    movies_fp_mean_by_year_likes = movies_fp_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    
    #gross
    movies_f_gross = gross_without_outliers.query('gender=="female"')
    movies_m_gross = gross_without_outliers.query('gender=="male"')
    movies_w_gross = gross_without_outliers.query('race_simple=="White"')
    movies_p_gross = gross_without_outliers.query('race_simple=="POC"')
    
    movies_fw_gross = movies_w_gross.query('gender=="female"')
    movies_mp_gross = movies_p_gross.query('gender=="male"')
    movies_mw_gross = movies_m_gross.query('race_simple=="White"')
    movies_fp_gross = movies_f_gross.query('race_simple=="POC"')
    
    movies_fw_gross_year = movies_fw_gross.groupby(by='title_year', sort=True)
    movies_mp_gross_year = movies_mp_gross.groupby(by='title_year', sort=True)
    movies_mw_gross_year = movies_mw_gross.groupby(by='title_year', sort=True)
    movies_fp_gross_year = movies_fp_gross.groupby(by='title_year', sort=True)
    
    #sum
    movies_fw_sum_by_year_gross = movies_fw_gross_year.aggregate({'gross': 'sum'}).reset_index()
    movies_mp_sum_by_year_gross = movies_mp_gross_year.aggregate({'gross': 'sum'}).reset_index()
    movies_mw_sum_by_year_gross = movies_mw_gross_year.aggregate({'gross': 'sum'}).reset_index()
    movies_fp_sum_by_year_gross = movies_fp_gross_year.aggregate({'gross': 'sum'}).reset_index()
    
    #average
    movies_fw_mean_by_year_gross = movies_fw_gross_year.aggregate({'gross': 'mean'}).reset_index()
    movies_mp_mean_by_year_gross = movies_mp_gross_year.aggregate({'gross': 'mean'}).reset_index()
    movies_mw_mean_by_year_gross = movies_mw_gross_year.aggregate({'gross': 'mean'}).reset_index()
    movies_fp_mean_by_year_gross = movies_fp_gross_year.aggregate({'gross': 'mean'}).reset_index()
    
else:
    movies_f = movies_dataframe_noDuplicates.query('gender=="female"')
    movies_m = movies_dataframe_noDuplicates.query('gender=="male"')
    movies_w = movies_dataframe_noDuplicates.query('race_simple=="White"')
    movies_p = movies_dataframe_noDuplicates.query('race_simple=="POC"')
    
    movies_mw = movies_m.query('race_simple=="White"')
    movies_mp = movies_m.query('race_simple=="POC"')
    movies_fw = movies_f.query('race_simple=="White"')
    movies_fp = movies_f.query('race_simple=="POC"')
    
    movies_mw_year = movies_mw.groupby(by='title_year', sort=True)
    movies_mp_year = movies_mp.groupby(by='title_year', sort=True)
    movies_fw_year = movies_fw.groupby(by='title_year', sort=True)
    movies_fp_year = movies_fp.groupby(by='title_year', sort=True)

    # group and aggregate by title year - sum
    movies_fw_sum_by_year = movies_fw_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()
    movies_mw_sum_by_year = movies_mw_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()
    movies_fp_sum_by_year = movies_fp_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()
    movies_mp_sum_by_year = movies_mp_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()

    # group and aggregate by title year - average
    movies_fw_mean_by_year = movies_fw_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()
    movies_mw_mean_by_year = movies_mw_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()
    movies_fp_mean_by_year = movies_fp_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()
    movies_mp_mean_by_year = movies_mp_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()

In [ ]:
# Adding to the db if no values for a year are available
# Wanted len: 8
if(remove_statistical_outliers):
    dfs = {'movies_fw_sum_by_year_likes': movies_fw_sum_by_year_likes, 
           'movies_mp_sum_by_year_likes': movies_mp_sum_by_year_likes, 
           'movies_mw_sum_by_year_likes': movies_mw_sum_by_year_likes, 
           'movies_fp_sum_by_year_likes': movies_fp_sum_by_year_likes, 
           'movies_fw_sum_by_year_likes': movies_fw_mean_by_year_likes, 
           'movies_mp_sum_by_year_likes': movies_mp_mean_by_year_likes, 
           'movies_mw_sum_by_year_likes': movies_mw_mean_by_year_likes, 
           'movies_fp_sum_by_year_likes': movies_fp_mean_by_year_likes, 
           'movies_fw_sum_by_year_gross': movies_fw_sum_by_year_gross, 
           'movies_mp_sum_by_year_gross': movies_mp_sum_by_year_gross, 
           'movies_mw_sum_by_year_gross': movies_mw_sum_by_year_gross, 
           'movies_fp_sum_by_year_gross': movies_fp_sum_by_year_gross, 
           'movies_fw_sum_by_year_gross': movies_fw_mean_by_year_gross, 
           'movies_mp_sum_by_year_gross': movies_mp_mean_by_year_gross, 
           'movies_mw_sum_by_year_gross': movies_mw_mean_by_year_gross, 
           'movies_fp_sum_by_year_gross': movies_fp_mean_by_year_gross}
else:
    dfs = {'movies_mw_sum_by_year': movies_mw_sum_by_year, 
           'movies_fp_sum_by_year': movies_fp_sum_by_year, 
           'movies_fw_sum_by_year': movies_fw_sum_by_year, 
           'movies_mp_sum_by_year': movies_mp_sum_by_year, 
           'movies_mw_mean_by_year': movies_mw_mean_by_year, 
           'movies_fp_mean_by_year': movies_fp_mean_by_year, 
           'movies_fw_mean_by_year': movies_fw_mean_by_year, 
           'movies_mp_mean_by_year': movies_mp_mean_by_year}

def correctDf(d):
    d = dfs[d]
    if(len(d) == 8):
        return d
    c=list(d)
    years=[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
    existing_years=[]
    for index, row in d.iterrows():
        existing_years.append(row['title_year'])
    for y in years:
        if (y not in existing_years):
            d = d.append(pd.DataFrame([[y,0,0]], columns=c))
    return d

dfs_corrected_subcats = {k: correctDf(k).sort_values(by='title_year') for k, v in dfs.items()}

dfs_corrected_subcats['movies_mp_mean_by_year']

In [ ]:
#add subcategory to the datasets

if(remove_statistical_outliers):
    fp = 'female, POC'
    fw = 'female, white'
    mw = 'male, white'
    mp = 'male, POC'
    pd.options.mode.chained_assignment = None #disable default warning that is a false positive here https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
    dfs_corrected_subcats['movies_fp_sum_by_year_gross']['subcat'] = fp
    dfs_corrected_subcats['movies_fp_mean_by_year_gross']['subcat'] = fp
    dfs_corrected_subcats['movies_mp_sum_by_year_gross']['subcat'] = mp
    dfs_corrected_subcats['movies_mp_mean_by_year_gross']['subcat'] = mp
    dfs_corrected_subcats['movies_mw_sum_by_year_gross']['subcat'] = mw
    dfs_corrected_subcats['movies_mw_mean_by_year_gross']['subcat'] = mw
    dfs_corrected_subcats['movies_fw_sum_by_year_gross']['subcat'] = fw 
    dfs_corrected_subcats['movies_fw_mean_by_year_gross']['subcat'] = fw
    dfs_corrected_subcats['movies_fp_sum_by_year_likes']['subcat'] = fp
    dfs_corrected_subcats['movies_fp_mean_by_year_likes']['subcat'] = fp
    dfs_corrected_subcats['movies_mp_sum_by_year_likes']['subcat'] = mp
    dfs_corrected_subcats['movies_mp_mean_by_year_likes']['subcat'] = mp
    dfs_corrected_subcats['movies_mw_sum_by_year_likes']['subcat'] = mw
    dfs_corrected_subcats['movies_mw_mean_by_year_likes']['subcat'] = mw
    dfs_corrected_subcats['movies_fw_sum_by_year_likes']['subcat'] = fw
    dfs_corrected_subcats['movies_fw_mean_by_year_likes']['subcat'] = fw
else:
    pd.options.mode.chained_assignment = None #disable default warning that is a false positive here https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
    dfs_corrected_subcats['movies_fw_sum_by_year']['subcat'] = 'female'
    dfs_corrected_subcats['movies_fw_mean_by_year']['subcat'] = 'female'
    dfs_corrected_subcats['movies_mw_sum_by_year']['subcat'] = 'male'
    dfs_corrected_subcats['movies_mw_mean_by_year']['subcat'] = 'male'
    dfs_corrected_subcats['movies_fp_sum_by_year']['subcat'] = 'white' 
    dfs_corrected_subcats['movies_fp_mean_by_year']['subcat'] = 'white'
    dfs_corrected_subcats['movies_mp_sum_by_year']['subcat'] = 'POC' 
    dfs_corrected_subcats['movies_mp_mean_by_year']['subcat'] = 'POC'


In [ ]:
# Then, get data for top categories: male (M) - female (F), white (W) - Person of Color (P)
if(remove_statistical_outliers):
    #likes
    movies_f_likes_year = movies_f_likes.groupby(by='title_year', sort=True)
    movies_m_likes_year = movies_m_likes.groupby(by='title_year', sort=True)
    movies_w_likes_year = movies_w_likes.groupby(by='title_year', sort=True)
    movies_p_likes_year = movies_p_likes.groupby(by='title_year', sort=True)
    
    #sum
    movies_f_sum_by_year_likes = movies_f_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    movies_m_sum_by_year_likes = movies_m_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    movies_w_sum_by_year_likes = movies_w_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    movies_p_sum_by_year_likes = movies_p_likes_year.aggregate({'movie_facebook_likes': 'sum'}).reset_index()
    
    #average
    movies_f_mean_by_year_likes = movies_f_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    movies_m_mean_by_year_likes = movies_m_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    movies_w_mean_by_year_likes = movies_w_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    movies_p_mean_by_year_likes = movies_p_likes_year.aggregate({'movie_facebook_likes': 'mean'}).reset_index()
    
    #gross
    movies_f_gross_year = movies_f_gross.groupby(by='title_year', sort=True)
    movies_m_gross_year = movies_m_gross.groupby(by='title_year', sort=True)
    movies_w_gross_year = movies_w_gross.groupby(by='title_year', sort=True)
    movies_p_gross_year = movies_p_gross.groupby(by='title_year', sort=True)
    
    #sum
    movies_f_sum_by_year_gross = movies_f_gross_year.aggregate({'gross': 'sum'}).reset_index()
    movies_m_sum_by_year_gross = movies_m_gross_year.aggregate({'gross': 'sum'}).reset_index()
    movies_w_sum_by_year_gross = movies_w_gross_year.aggregate({'gross': 'sum'}).reset_index()
    movies_p_sum_by_year_gross = movies_p_gross_year.aggregate({'gross': 'sum'}).reset_index()
    
    #average
    movies_f_mean_by_year_gross = movies_f_gross_year.aggregate({'gross': 'mean'}).reset_index()
    movies_m_mean_by_year_gross = movies_m_gross_year.aggregate({'gross': 'mean'}).reset_index()
    movies_w_mean_by_year_gross = movies_w_gross_year.aggregate({'gross': 'mean'}).reset_index()
    movies_p_mean_by_year_gross = movies_p_gross_year.aggregate({'gross': 'mean'}).reset_index()
    
else:
    movies_w_year = movies_w.groupby(by='title_year', sort=True)
    movies_m_year = movies_m.groupby(by='title_year', sort=True)
    movies_f_year = movies_f.groupby(by='title_year', sort=True)
    movies_p_year = movies_p.groupby(by='title_year', sort=True)
    
    # group and aggregate by title year
    # https://stackoverflow.com/questions/46826773/how-can-i-merge-rows-by-same-value-in-a-column-in-pandas-with-aggregation-func
    #sum
    movies_f_sum_by_year = movies_f_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()
    movies_m_sum_by_year = movies_p_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()
    movies_w_sum_by_year = movies_w_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()
    movies_p_sum_by_year = movies_p_year.aggregate({'gross': 'sum', 'movie_facebook_likes': 'sum'}).reset_index()

    #average
    movies_f_mean_by_year = movies_f_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()
    movies_m_mean_by_year = movies_m_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()
    movies_w_mean_by_year = movies_w_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()
    movies_p_mean_by_year = movies_p_year.aggregate({'gross': 'mean', 'movie_facebook_likes': 'mean'}).reset_index()


In [ ]:
# Adding to the db if no values for a year are available
# Wanted len: 8
if(remove_statistical_outliers):
    dfs = {'movies_w_sum_by_year_likes': movies_w_sum_by_year_likes, 
           'movies_p_sum_by_year_likes': movies_p_sum_by_year_likes, 
           'movies_m_sum_by_year_likes': movies_m_sum_by_year_likes, 
           'movies_f_sum_by_year_likes': movies_f_sum_by_year_likes, 
           'movies_w_sum_by_year_likes': movies_w_mean_by_year_likes, 
           'movies_p_sum_by_year_likes': movies_p_mean_by_year_likes, 
           'movies_m_sum_by_year_likes': movies_m_mean_by_year_likes, 
           'movies_f_sum_by_year_likes': movies_f_mean_by_year_likes, 
           'movies_w_sum_by_year_gross': movies_w_sum_by_year_gross, 
           'movies_p_sum_by_year_gross': movies_p_sum_by_year_gross, 
           'movies_m_sum_by_year_gross': movies_m_sum_by_year_gross, 
           'movies_f_sum_by_year_gross': movies_f_sum_by_year_gross, 
           'movies_w_sum_by_year_gross': movies_w_mean_by_year_gross, 
           'movies_p_sum_by_year_gross': movies_p_mean_by_year_gross, 
           'movies_m_sum_by_year_gross': movies_m_mean_by_year_gross, 
           'movies_f_sum_by_year_gross': movies_f_mean_by_year_gross}
else:
    dfs = {'movies_w_sum_by_year': movies_w_sum_by_year, 
           'movies_p_sum_by_year': movies_p_sum_by_year, 
           'movies_f_sum_by_year': movies_f_sum_by_year, 
           'movies_m_sum_by_year': movies_m_sum_by_year, 
           'movies_w_mean_by_year': movies_w_mean_by_year, 
           'movies_p_mean_by_year': movies_p_mean_by_year, 
           'movies_f_mean_by_year': movies_f_mean_by_year, 
           'movies_m_mean_by_year': movies_m_mean_by_year}

dfs_corrected_topcats = {k: correctDf(k).sort_values(by='title_year') for k, v in dfs.items()}

dfs_corrected_topcats['movies_p_mean_by_year']

In [ ]:
#merge all datasets from subcats together for later use
movies_with_subcat = pd.concat(dfs_corrected_subcats) #http://pandas.pydata.org/pandas-docs/stable/merging.html

In [ ]:
# Part 2: Bar diagrams for overall development

# The following function creates a bar chart with two y-Axis and one x-Axis.
def vis_bar(title, x_axis, y_axis1, y_label1, y_axis2, y_label2, **keyword_parameters):
    ind = np.arange(len(x_axis))
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    ax2 = ax.twinx() #https://stackoverflow.com/questions/14762181/adding-a-y-axis-label-to-secondary-y-axis-in-matplotlib#14762601

    #https://matplotlib.org/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
    rects1 = ax.bar(ind - width/2, y_axis1, width, yerr=0, xerr=0,
                color='SkyBlue', label=y_label1)
    rects2 = ax2.bar(ind + width/2, y_axis2, width, yerr=0, xerr=0,
                color='IndianRed', label=y_label2)
    
    ax.set_ylabel(y_label1)
    ax2.set_ylabel(y_label2)
    
    ax.legend(bbox_to_anchor=(1.3, 1), loc=2, borderaxespad=0.)
    ax2.legend(bbox_to_anchor=(1.3, 0.9), loc=2, borderaxespad=0.)
    
    ax.set_xticks(ind)
    ax.set_xticklabels(x_axis)
    
    plt.title(label=title, pad=20.0, loc='left')
    
    #https://stackoverflow.com/questions/14017996/is-there-a-way-to-pass-optional-parameters-to-a-function
    if('horizontal_xlabels' not in keyword_parameters):
        #https://stackoverflow.com/questions/12273712/matplotlib-xticks-labels-overlap
        plt.setp(ax.get_xticklabels(), fontsize=10, rotation='vertical')
    
    if('filename' not in keyword_parameters):
        path='graphs/' + title.replace(" ", "").replace("*", "").replace(":", "") + '.png'
    else:
        path='graphs/' + keyword_parameters['filename'] + '.png'
    
    fig.savefig(path, transparent=True, bbox_inches='tight') #save the graph
    print('saved to ', path)
    
    plt.show()

In [ ]:
def with_outliers_create_bar_plots_development_totals():
    # Creating bar charts for all top categories
    vis_bar(title='Development of Totals: Films with lead actors* who are people of color', 
            filename='bar_year_gross_likes_total_p', 
            x_axis=dfs_corrected_topcats['movies_p_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_p_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_p_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Totals: Films with white lead actors*', 
            filename='bar_year_gross_likes_total_w', 
            x_axis=dfs_corrected_topcats['movies_w_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_w_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_w_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Totals: Films with lead actresses', 
            filename='bar_year_gross_likes_total_f', 
            x_axis=dfs_corrected_topcats['movies_f_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_f_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_f_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Totals: Films with lead actors', 
            filename='bar_year_gross_likes_total_m', 
            x_axis=dfs_corrected_topcats['movies_m_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_m_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_m_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)
    
    # Creating bar charts for all sub categories
    vis_bar(title='Development of Totals: Films with lead actresses who are women of color', 
            filename='bar_year_gross_likes_total_fp', 
            x_axis=dfs_corrected_subcats['movies_fp_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_fp_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_fp_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Totals: Films with white lead actresses', 
            filename='bar_year_gross_likes_total_fw', 
            x_axis=dfs_corrected_subcats['movies_fw_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_fw_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_fw_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Totals: Films with lead actors who are men of color', 
            filename='bar_year_gross_likes_total_mp', 
            x_axis=dfs_corrected_subcats['movies_mp_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_mp_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_mp_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Totals: Films with white lead actors', 
            filename='bar_year_gross_likes_total_mw', 
            x_axis=dfs_corrected_subcats['movies_mw_sum_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_mw_sum_by_year']['gross'], 
            y_label1='Total Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_mw_sum_by_year']['movie_facebook_likes'], 
            y_label2='Total Facebook Likes', 
            horizontal_xlabels=True)

if(not remove_statistical_outliers):
    with_outliers_create_bar_plots_development_totals()

In [ ]:
def with_outliers_create_bar_plots_development_means():
    # Creating bar charts for all top categories
    vis_bar(title='Development of Averages: Films with lead actors* who are people of color', 
            filename='bar_year_gross_likes_mean_p', 
            x_axis=dfs_corrected_topcats['movies_p_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_p_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_p_mean_by_year']['movie_facebook_likes'],
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Averages: Films with white lead actors*', 
            filename='bar_year_gross_likes_mean_w', 
            x_axis=dfs_corrected_topcats['movies_w_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_w_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_w_mean_by_year']['movie_facebook_likes'], 
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Averages: Films with lead actresses', 
            filename='bar_year_gross_likes_mean_f', 
            x_axis=dfs_corrected_topcats['movies_f_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_f_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_f_mean_by_year']['movie_facebook_likes'], 
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Averages: Films with lead actors', 
            filename='bar_year_gross_likes_mean_m', 
            x_axis=dfs_corrected_topcats['movies_m_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_topcats['movies_m_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_topcats['movies_m_mean_by_year']['movie_facebook_likes'], 
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)
    
    # Creating bar charts for all sub categories
    vis_bar(title='Development of Averages: Films with lead actresses who are women of color', 
            filename='bar_year_gross_likes_mean_fp', 
            x_axis=dfs_corrected_subcats['movies_fp_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_fp_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_fp_mean_by_year']['movie_facebook_likes'], 
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Averages: Films with white lead actresses', 
            filename='bar_year_gross_likes_mean_fw', 
            x_axis=dfs_corrected_subcats['movies_fw_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_fw_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_fw_mean_by_year']['movie_facebook_likes'], 
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Averages: Films with lead actors who are men of color', 
            filename='bar_year_gross_likes_mean_mp', 
            x_axis=dfs_corrected_subcats['movies_mp_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_mp_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_mp_mean_by_year']['movie_facebook_likes'], 
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)
    vis_bar(title='Development of Averages: Films with white lead actors', 
            filename='bar_year_gross_likes_mean_mw', 
            x_axis=dfs_corrected_subcats['movies_mw_mean_by_year']['title_year'], 
            y_axis1=dfs_corrected_subcats['movies_mw_mean_by_year']['gross'], 
            y_label1='Average Domestic Gross Revenue', 
            y_axis2=dfs_corrected_subcats['movies_mw_mean_by_year']['movie_facebook_likes'], 
            y_label2='Average Facebook Likes', 
            horizontal_xlabels=True)

if(not remove_statistical_outliers):
    with_outliers_create_bar_plots_development_means()

In [ ]:
#Part 3: calculate total and average domestic gross revenue and fb likes for each top category and sub category 
#and make a dataframe from it
#columns: CATEGORY | total fb likes | average fb likes | total domestic gross revenue | average domestic gross revenue 
#categories: 
# df 1 g: M, F, 
# df 2 r: W, P; 
# df 3 gr: MW, MP, FW, FP
def sumAndAvg(col1, col2):
    row = [0,0,0,0]
    i = 0
    for col in [col1, col2]:
        row[i] = col.sum()
        i+=1
        row[i] = col.mean()
        i+=1
    return row

def get_sum_and_averages():
    dfs = [movies_m, movies_f, movies_w, movies_p, movies_mw, movies_mp, movies_fw, movies_fp]

    columns=['total_gross', 'average_gross', 'total_movie_facebook_likes', 'average_movie_facebook_likes']
    categories = ['m', 'f', 'w', 'p', 'mw', 'mp', 'fw', 'fp']
    rows = [sumAndAvg(df['gross'], df['movie_facebook_likes']) for df in dfs]

    return pd.DataFrame(rows, columns=columns).assign(category=categories)

In [ ]:
# for each df in topcats and subcats
# 

In [ ]:
#Part 4: Get own dataframes for each category: gender, race, race and gender

if(remove_statistical_outliers):
    print('hi')
    '''
    Available dfs:
    movies_f_likes, movies_m_likes, movies_w_likes, movies_p_likes
    movies_f_gross, movies_m_gross, movies_w_gross, movies_p_gross

    movies_fw_likes, movies_mp_likes, movies_mw_likes, movies_fp_likes
    movies_fw_gross, movies_mp_gross, movies_mw_gross, movies_fp_gross
    
    How to bring it in the form of g, r, gr below? needed:
    columns=['total_gross', 'average_gross', 'total_movie_facebook_likes', 'average_movie_facebook_likes']
    '''
else:
    statisticalResults = get_sum_and_averages()
    g = statisticalResults[(statisticalResults.category == 'm') | (statisticalResults.category == 'f')]
    r = statisticalResults[(statisticalResults.category == 'w') | (statisticalResults.category == 'p')]
    gr = statisticalResults[(statisticalResults.category == 'mw') | (statisticalResults.category == 'fw') | (statisticalResults.category == 'mp') | (statisticalResults.category == 'fp')]
    print(gr)

In [ ]:
def create_bar_plots_comparison_totals():
    #Part 5a: Compare total dgr & fb likes per categories with bar diagrams
    vis_bar(title='Comparison of Totals: Films by gender of the lead actor*', filename='bar_g_gross_likes_total', x_axis=g['category'], y_axis1=g['total_gross'], y_label1='Total Domestic Gross Revenue', y_axis2=g['total_movie_facebook_likes'], y_label2='Total Facebook Likes', horizontal_xlabels=True)
    vis_bar(title='Comparison of Totals: Films by race of the lead actor*', filename='bar_r_gross_likes_total', x_axis=r['category'], y_axis1=r['total_gross'], y_label1='Total Domestic Gross Revenue', y_axis2=r['total_movie_facebook_likes'], y_label2='Total Facebook Likes', horizontal_xlabels=True)

    vis_bar(title='Comparison of Totals: Films by gender and race of the lead actor*', filename='bar_gr_gross_likes_total', x_axis=gr['category'], y_axis1=gr['total_gross'], y_label1='Total Domestic Gross Revenue', y_axis2=gr['total_movie_facebook_likes'], y_label2='Total Facebook Likes', horizontal_xlabels=True)

def create_bar_plots_comparison_averages():
    #Part 6a: Compare average dgr & fb likes per categories
    vis_bar(title='Comparison of Averages: Films by gender of the lead actor*', filename='bar_g_gross_likes_average', x_axis=g['category'], y_axis1=g['average_gross'], y_label1='Average Domestic Gross Revenue', y_axis2=g['average_movie_facebook_likes'], y_label2='Average Facebook Likes', horizontal_xlabels=True)
    vis_bar(title='Comparison of Averages: Films by race of the lead actor*', filename='bar_r_gross_likes_average', x_axis=r['category'], y_axis1=r['average_gross'], y_label1='Average Domestic Gross Revenue', y_axis2=r['average_movie_facebook_likes'], y_label2='Average Facebook Likes', horizontal_xlabels=True)

    vis_bar(title='Comparison of Averages: Films by gender and race of the lead actor*', filename='bar_gr_gross_likes_average', x_axis=gr['category'], y_axis1=gr['average_gross'], y_label1='Average Domestic Gross Revenue', y_axis2=gr['average_movie_facebook_likes'], y_label2='Average Facebook Likes', horizontal_xlabels=True)

create_bar_plots_comparison_totals()
create_bar_plots_comparison_averages()

In [ ]:
#Part 7a: Function for making a pie chart
#https://pythonspot.com/matplotlib-pie-chart/
#https://matplotlib.org/gallery/pie_and_polar_charts/pie_and_donut_labels.html#sphx-glr-gallery-pie-and-polar-charts-pie-and-donut-labels-py
def vis_pie(title, legend, value, label, **keyword_parameters):
    fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))

    wedges, texts, autotexts = ax.pie(value, autopct='%1.1f%%',
                                      textprops=dict(color="w"))

    ax.legend(wedges, legend,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=8, weight="bold")

    ax.set_title(label)

    if('filename' not in keyword_parameters):
        path='graphs/' + title.replace(" ", "").replace("*", "").replace(":", "") + '.png'
    else:
        path='graphs/' + keyword_parameters['filename'] + '.png'
    
    fig.savefig(path, transparent=True, bbox_inches='tight') #save the graph
    print('saved to ', path)
    
    plt.show()

In [ ]:
#Part 7: Compare total dgr, fb likes, and IMDB Score per categories with pie diagrams
def create_pie_plots():
    vis_pie(title='Comparison of Totals: Film Revenue by gender of the lead actor*', filename='pie_g_gross_total', legend=g['category'], value=g['total_gross'], label='Total Domestic Gross Revenue')
    vis_pie(title='Comparison of Totals: Film Likes by gender of the lead actor*', filename='pie_g_likes_total', legend=g['category'], value=g['total_movie_facebook_likes'], label='Total Facebook Likes')
    
    vis_pie(title='Comparison of Totals: Film Revenue by race of the lead actor*', filename='pie_r_gross_total', legend=r['category'], value=r['total_gross'], label='Total Domestic Gross Revenue')
    vis_pie(title='Comparison of Totals: Film Likes by race of the lead actor*', filename='pie_r_likes_total', legend=r['category'], value=r['total_movie_facebook_likes'], label='Total Facebook Likes')
    
    vis_pie(title='Comparison of Totals: Film Revenue by gender and race of the lead actor*', filename='pie_gr_gross_total', legend=gr['category'], value=gr['total_gross'], label='Total Domestic Gross Revenue')
    vis_pie(title='Comparison of Totals: Film Likes by gender and race of the lead actor*', filename='pie_gr_likes_total', legend=gr['category'], value=gr['total_movie_facebook_likes'], label='Total Facebook Likes')

In [ ]:
#Part 8: Create splatter diagrams
def vis_splat(x_axis, y_axis, ylabel, title, **keyword_parameters):
    fig, ax = plt.subplots()
    plt.plot(x_axis, y_axis, 'o', alpha=.5, label=ylabel)
    #https://matplotlib.org/users/legend_guide.html
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title(label=title, pad=20.0, loc='left')
    
    if('filename' not in keyword_parameters):
        path='graphs/' + title.replace(" ", "").replace("*", "").replace(":", "") + '.png'
    else:
        path='graphs/' + keyword_parameters['filename'] + '.png'
    
    fig.savefig(path, transparent=True, bbox_inches='tight') #save the graph
    print('saved to ', path)
    
    plt.show()


In [ ]:
#Correct diagrams for non-outlier data
def correctDf8(d):
    d = dfs[d]
    if(len(d) == 8):
        return d
    c=list(d)
    years=[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
    existing_years=[]
    for index, row in d.iterrows():
        existing_years.append(row['title_year'])
    for y in years:
        if (y not in existing_years):
            d = d.append(pd.DataFrame([[0,0,y,0,0,0,0,0]], columns=c))
    return d

dfs = {'movies_f': movies_f, 
           'movies_m': movies_m, 
           'movies_p': movies_p, 
           'movies_w': movies_w, 
           'movies_mw': movies_mw, 
           'movies_fp': movies_fp, 
           'movies_fw': movies_fw, 
           'movies_mp': movies_mp}

dfs_corrected_upper = {k: correctDf8(k).sort_values(by='title_year') for k, v in dfs.items()}

dfs_corrected_upper['movies_p']

In [ ]:
#TODO: call by dfs_corrected_upper['movies_p']
def create_scatter_plots():
    # a) x = year of movie, y = gross (each movie) (for female)
    # x = year of movie, y = gross (each movie) (for male)
    # x = year of movie, y = gross (each movie) (for white)
    # x = year of movie, y = gross (each movie) (for POC)
    # x = year of movie, y = gross (each movie) (for mw, mp, fw, fp)
    vis_splat(movies_f["title_year"], movies_f["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a lead actress", filename="scatter_year_gross_f")
    vis_splat(movies_m["title_year"], movies_m["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a lead actor", filename="scatter_year_gross_m")
    vis_splat(movies_w["title_year"], movies_w["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a white lead actor*", filename="scatter_year_gross_w")
    vis_splat(movies_p["title_year"], movies_p["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a lead actor* who is a person of color", filename="scatter_year_gross_n")
    
    vis_splat(movies_mw["title_year"], movies_mw["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a white lead actor", filename="scatter_year_gross_mw")
    vis_splat(movies_mp["title_year"], movies_mp["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a lead actor who is a man of color", filename="scatter_year_gross_mp")
    vis_splat(movies_fw["title_year"], movies_fw["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a white lead actress", filename="scatter_year_gross_fw")
    vis_splat(movies_fp["title_year"], movies_fp["gross"], "Gross Domestic Revenue", "Development: Gross Domestic Revenue of each movie with a lead actress who is a woman of color", filename="scatter_year_gross_fp")
    
    # b) x = year of movie, y = FB likes
    vis_splat(movies_f["title_year"], movies_f["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a lead actress", filename="scatter_year_likes_f")
    vis_splat(movies_m["title_year"], movies_m["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a lead actor", filename="scatter_year_likes_m")
    vis_splat(movies_w["title_year"], movies_w["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a white lead actor*", filename="scatter_year_likes_w")
    vis_splat(movies_p["title_year"], movies_p["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a POC lead actor*", filename="scatter_year_likes_p")
    
    vis_splat(movies_mw["title_year"], movies_mw["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a white lead actor", filename="scatter_year_likes_mw")
    vis_splat(movies_mp["title_year"], movies_mp["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a lead actor who is a man of color", filename="scatter_year_likes_mp")
    vis_splat(movies_fw["title_year"], movies_fw["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a white lead actress", filename="scatter_year_likes_fw")
    vis_splat(movies_fp["title_year"], movies_fp["movie_facebook_likes"], "Facebook Likes", "Development: Facebook likes of each movie with a lead actress who is a woman of color", filename="scatter_year_likes_fp")
    
    # c) x = gender of lead actor, y = gross (each movie)
    # d) x = gender, y = FB likes
    vis_splat(movies_dataframe_noDuplicates["gender"], movies_dataframe_noDuplicates["gross"], "Domestic Gross Revenue", "Comparison: Domestic Gross Revenue of each movie per gender", filename="scatter_g_gross")
    vis_splat(movies_dataframe_noDuplicates["gender"], movies_dataframe_noDuplicates["movie_facebook_likes"], "Facebook Likes", "Comparison: Facebook Likes of each movie per gender", filename="scatter_g_likes")
    
    # e) x = race, y = gross
    # e) x = race, y = fb likes
    vis_splat(movies_dataframe_noDuplicates["race_simple"], movies_dataframe_noDuplicates["gross"], "Domestic Gross Revenue", "Comparison: Domestic Gross Revenue of each movie per race", filename="scatter_r_gross")
    vis_splat(movies_dataframe_noDuplicates["race_simple"], movies_dataframe_noDuplicates["movie_facebook_likes"], "Facebook Likes", "Comparison: Facebook Likes of each movie per race", filename="scatter_r_likes")

    # f) x = gender and race, y = gross
    # g) x = gender and race, y = FB likes  
    # commented out because this doesn't have to run each time
    #TODO: this is sth else no or not?
    vis_splat(movies_with_subcat["subcat"], movies_with_subcat["gross"], "Domestic Gross Revenue", "Comparison: Domestic Gross Revenue of each movie per gender and race", filename="scatter_gr_gross")
    vis_splat(movies_with_subcat["subcat"], movies_with_subcat["movie_facebook_likes"], "Facebook Likes", "Comparison: Facebook Likes of each movie per gender and race", filename="scatter_gr_likes")

create_scatter_plots()

In [ ]:
# Part 9: Remove statistical outliers and create new graphs.
# Testing Zone. 
# For learning more about these outliers, in one of the upper cells we will remove the outliers testwise, to see how that plays into the graphs
# Following the recipe from https://www.wikihow.com/Calculate-Outliers
# 9a) Gross
'''
gross_q25 = movies_dataframe_noDuplicates['gross'].quantile(.25)
gross_q75 = movies_dataframe_noDuplicates['gross'].quantile(.75)
gross_interquart_range = gross_q75 - gross_q25
gross_outer_fence_lower = gross_q25 - (gross_interquart_range * 3)
gross_outer_fence_higher = gross_q75 + (gross_interquart_range * 3)
gross_major_outliers1 = movies_dataframe_noDuplicates.sort_values(by='gross').query('gross <='  + str(gross_outer_fence_lower))
gross_major_outliers2 = movies_dataframe_noDuplicates.sort_values(by='gross').query('gross >='  + str(gross_outer_fence_higher))
if(len(gross_major_outliers1) + len(gross_major_outliers2) > 0):
    print("Outliers below")
    print(gross_major_outliers1)
    print("Outliers above")
    print(gross_major_outliers2)
else:
    print("No statistical outliers found for gross revenue.")
'''

In [ ]:
# 9b) Likes
'''
likes_q25 = movies_dataframe_noDuplicates['movie_facebook_likes'].quantile(.25)
likes_q75 = movies_dataframe_noDuplicates['movie_facebook_likes'].quantile(.75)
likes_interquart_range = likes_q75 - likes_q25
likes_outer_fence_lower = likes_q25 - (likes_interquart_range * 3)
likes_outer_fence_higher = likes_q75 + (likes_interquart_range * 3)
likes_major_outliers1 = movies_dataframe_noDuplicates.sort_values(by='movie_facebook_likes').query('movie_facebook_likes <='  + str(likes_outer_fence_lower))
likes_major_outliers2 = movies_dataframe_noDuplicates.sort_values(by='movie_facebook_likes').query('movie_facebook_likes >='  + str(likes_outer_fence_higher))
if(len(likes_major_outliers1) + len(likes_major_outliers2) > 0):
    print("Outliers below")
    print(likes_major_outliers1)
    print("Outliers above")
    print(likes_major_outliers2)
else:
    print("No statistical outliers found for FB likes.")
'''